### Import the necessary libraries

In [ ]:
import pandas as pd
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from tqdm import tqdm

In [ ]:
# Create .env file within your directory and add your OpenAI API key
import openai, os
import sys
sys.path.append('../..')
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
llm_model = "gpt-4-turbo-preview"

### Use OpenAI's new text-embedding-3-large embeddings

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

### Load the vector database

In [ ]:
def vector_db():
    qdrant_client = QdrantClient(
        url = "http://localhost:6333",
        prefer_grpc=False
    )
    db = Qdrant(client=qdrant_client, embeddings=embeddings, collection_name="UNIFI_Vector_DB")
    return db

In [ ]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the integer value that corresponds to the value of the metric asked and nothing else.
"""
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

In [ ]:
def retrieval_qa_chain(llm, prompt, db):       
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain

In [ ]:
amkey = pd.read_csv('../AMKEY_GoldenStandard.csv')

In [ ]:
amkey_synonyms = pd.read_csv('../ActivityMetricsSynonyms.csv')

In [ ]:
company_groups = ["Absa", "Clicks", "Distell", "Oceana1&2", "Ssw", "Picknpay", "Impala", "Sasol"]

In [ ]:
company_group_name_for_inference =  {
    "Absa": "Absa",
    "Clicks": "Clicks Group Limited",
    "Distell": "Distell",
    "Oceana1&2": "Oceana Group",
    "Ssw": "Sibanye Stillwater",
    "Picknpay": "Pick n Pay",
    "Impala": "Impala",
    "Sasol": "Sasol"
}

In [ ]:
sample_submission_df = pd.DataFrame(columns=['ID', '2022_Value'])

In [ ]:
llm = ChatOpenAI(model_name=llm_model, temperature=0.1)

submission_iter = 0
    
vector_db_ = vector_db()

for company_name in tqdm(company_groups):
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, vector_db_)

    # Iterate through each activity metric
    for amkey_value, activity_metric in zip(amkey['AMKEY'], amkey['ActivityMetric']):
        # Construct ID and query
        company_or_group = company_name
        # Use the amkey_value and company_or_group to get the ClientMetric amkey_synonyms dataframe
        client_metric = ""
        if not amkey_synonyms.empty:
            client_metric_values = amkey_synonyms[(amkey_synonyms['AMKEY'] == amkey_value) & (amkey_synonyms['Group'] == company_or_group)]['ClientMetric'].values
            if len(client_metric_values) > 0:
                client_metric = client_metric_values[0]

        if client_metric:
            query = f"What is the value of {client_metric} for {company_group_name_for_inference[company_or_group]} in 2022?"
        else:
            query = f"What is the value of {activity_metric} for {company_group_name_for_inference[company_or_group]} in 2022?"
        ID = f"{amkey_value}_X_{company_or_group}"

        try:
            # Query the model for the response
            response = qa({'query': query})
            print(response)
            value = response['result']
        except Exception as e:
            print(f"An error occurred for company: {company_name} and activity metric: {activity_metric}")
            print(f"Error message: {e}")
            value = 0.0

        # Check if the ID exists in the sample_submission_df
        if ID in sample_submission_df['ID'].values:
            index = sample_submission_df[sample_submission_df['ID'] == ID].index[0]
            existing_value = sample_submission_df.loc[index, '2022_Value']
            if existing_value == "I don't know." and value != "I don't know.":
                # Update the value in sample_submission_df if it's not already set
                sample_submission_df.loc[index, '2022_Value'] = value
        else:
            # Add a new entry to the sample_submission_df
            sample_submission_df.loc[submission_iter] = [ID, value]
            submission_iter += 1

In [ ]:
sample_submission_df.to_csv('submission1.csv', index=False)